In [1]:
# 구글 드라이브 마운트
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 패키지 import

In [2]:
# 필요한 패키지 임포트
!pip install sentence-transformers
!pip install faiss-cpu

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import math
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import torch
import faiss
from sentence_transformers import SentenceTransformer, util
warnings.filterwarnings('ignore')

# 1-1. Import Data

In [3]:
data = pd.read_csv('/content/drive/MyDrive/ChecKHUMate/merge_domitory_data.csv')

# 1-2. Data preprocessing

In [4]:
def prepare_data(data):
    """
    주어진 데이터 프레임에서 필요한 정보를 추출하고 포맷팅하는 함수.

    Parameters:
        data (pd.DataFrame): 원본 데이터 프레임.

    Returns:
        user_data (pd.DataFrame): 사용자 데이터 프레임.
        wish_data (pd.DataFrame): 사용자의 선호에 관한 데이터 프레임.
    """
    # 사용자 데이터 선택
    user_data_columns = [
        'user_id', 'domitory', 'age', 'student_id', 'gender', 'major',
        'bedtime', 'clean_duration', 'smoke', 'alcohol', 'mbti', 'one_sentence'
    ]
    user_data = data[user_data_columns]
    user_data = user_data.set_index('user_id')

    # 사용자 선호 데이터 선택
    wish_data_columns = [
        'user_id', 'wish_domitory', 'wish_age', 'wish_student_id', 'wish_gender',
        'wish_major', 'wish_bedtime', 'wish_clean_duration', 'wish_smoke',
        'wish_alcohol', 'wish_mbti'
    ]
    wish_data = data[wish_data_columns]
    wish_data = wish_data.set_index('user_id')

    return user_data, wish_data

In [5]:
user_data, wish_data = prepare_data(data)

In [6]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

def encode_and_concat_to_array(dataframe, column_name):
    """
    원핫 인코딩을 수행하고 원본 데이터프레임에 결과를 추가한 후 전체를 numpy 배열로 반환합니다.

    Args:
    dataframe (pd.DataFrame): 원본 데이터를 포함한 데이터프레임.
    column_name (str): 원핫 인코딩을 적용할 열의 이름.

    Returns:
    numpy.ndarray: 원핫 인코딩된 결과와 원본 데이터가 결합된 numpy 배열.
    """
    # 원핫 인코더 초기화 및 피팅
    encoder = OneHotEncoder(sparse_output=False)  # 바로 numpy array 반환 설정
    encoded_data = encoder.fit_transform(dataframe[[column_name]])

    # 원핫 인코딩 결과를 DataFrame으로 변환
    encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out([column_name]))

    # 데이터프레임의 인덱스 리셋
    dataframe = dataframe.reset_index(drop=True)
    encoded_df = encoded_df.reset_index(drop=True)

    # 원본 데이터와 원핫 인코딩된 데이터를 결합
    new_dataframe = pd.concat([dataframe, encoded_df], axis=1)

    # 인코딩된 열 제거
    new_dataframe = new_dataframe.drop(column_name, axis=1)

    return new_dataframe

In [7]:
user_df = encode_and_concat_to_array(user_data, 'mbti')
wish_df = encode_and_concat_to_array(wish_data, 'wish_mbti')

In [8]:
# Faiss 유사도 계산을 위한 데이터 전처리

# 1) user_data에서 one_sentence 데이터 drop
# 0행: 'user_id', 1행:'domitory', 2행: 'age', 3행: 'student_id', 4행: 'gender', 5행: 'major', 6행: 'bedtime', 7행: 'clean_duration', 8행:'smoke',
# 9행: 'alcohol', 10행: 'one_sentence'

# one_sentence drop
user_df_a = user_df.drop(columns = 'one_sentence')

# 2. Data Filtering & Modeling

## 2-1 Faiss 유사도

In [9]:
import numpy as np
import faiss

# 각 사용자의 wish_data를 기반으로 user_data에서 자기 자신을 제외한 상위 k개의 가장 유사한 사용자들을 찾아낸 결과를 배열로 반환합니다.
class Recommender_without_sentence:
    def __init__(self, user_data, wish_data):
        self.user_data = user_data
        self.wish_data = wish_data
        self.index = None
        self.d = user_data.shape[1]

    def build_index(self):
        """FAISS 인덱스를 생성하고 유저 데이터를 추가합니다."""
        self.index = faiss.IndexFlatL2(self.d)
        self.index.add(self.user_data.astype('float32'))

    def find_roommates(self, k=10):
        """각 유저의 wish_data에 대해 가장 유사한 유저를 찾되, 자기 자신은 제외하고, -1 인덱스가 발생한 경우도 기록합니다."""
        if self.index is None:
            self.build_index()

        all_results = []  # 결과를 저장할 배열
        invalid_indices_info = []  # -1 인덱스가 발생한 정보를 저장할 배열

        for i in range(self.wish_data.shape[0]):
            # 자기 자신을 포함하여 k+1개의 결과를 검색
            distances, indices = self.index.search(self.wish_data[i:i+1].astype('float32'), k+1)

            # 자기 자신의 인덱스와 -1을 제외
            valid_indices = indices[0][(indices[0] != i) & (indices[0] != -1)]

            # -1이 발생한 경우 기록
            if np.any(indices[0] == -1):
                invalid_indices_info.append((i, list(indices[0])))

            # k개의 결과만 반환
            all_results.append(valid_indices[:k])

        # 결과와 -1 정보 모두 반환
        return np.array(all_results)

In [10]:
recommender = Recommender_without_sentence(user_df_a, wish_df)
roommate_recommendations = recommender.find_roommates()

In [11]:
roommate_recommendations

array([[ 50,  55,  56, ...,  51,  22,  23],
       [ 51,   0,   3, ...,  22,  23,  26],
       [ 52,  32,  62, ...,  29,   3,   9],
       ...,
       [ 47,  32,  55, ...,  53,  62,   9],
       [ 48,  56,  62, ...,  38,  60,  64],
       [ 49,  10, 112, ...,  15,  14,  37]])

In [12]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import pandas as pd

class IntroductionRecommender:
    def __init__(self, user_data):
        self.user_data = user_data
        self.embedder = SentenceTransformer('jhgan/ko-sroberta-multitask')
        self.index = None
        self.id_index = np.array(self.user_data.index)
        self.embeddings = None
        self.user_data['one_sentence'] = user_data['one_sentence'].fillna('No introduction provided').astype(str)

    def create_faiss_index(self):
        self.embeddings = self.embedder.encode(self.user_data['one_sentence'].tolist(), convert_to_tensor=False)

        normalized_embeddings = self.embeddings.copy()
        faiss.normalize_L2(normalized_embeddings)

        d = self.embeddings.shape[1]
        index_flat = faiss.IndexFlatIP(d)
        self.index = faiss.IndexIDMap(index_flat)
        self.index.add_with_ids(normalized_embeddings, self.id_index)

    def find_similar_introductions(self, result_indices, k=10):
        if self.index is None:
            raise ValueError("FAISS index is not built. Please run create_faiss_index method first.")

        all_group_similarities = []

        for base_index, indices in enumerate(result_indices):
            if base_index >= len(self.user_data):
                continue

            base_vector = self.embeddings[base_index].reshape(1, -1)
            compare_vectors = self.embeddings[indices]

            D, I = self.index.search(base_vector, len(self.user_data))
            distances = [(I[0][i], D[0][i]) for i in range(len(I[0])) if I[0][i] in indices]

            distances = sorted(distances, key=lambda x: x[1], reverse=True)
            group_similarities = [idx for idx, _ in distances[:k]]
            all_group_similarities.append(group_similarities)

        return all_group_similarities

In [13]:
recommender = IntroductionRecommender(user_df)
recommender.create_faiss_index()
similarities = recommender.find_similar_introductions(roommate_recommendations)

In [14]:
roommate_recommendations[0]

array([50, 55, 56, 48, 62,  9, 47, 51, 22, 23])

In [15]:
print(similarities[0])

[56, 9, 51, 22, 55, 23, 62, 48, 47, 50]


In [16]:
user_data.iloc[0]

domitory                          0
age                               0
student_id                        0
gender                            1
major                             2
bedtime                           2
clean_duration                    0
smoke                             0
alcohol                           0
mbti                           ENTJ
one_sentence      매일 아침에 운동을 즐겨합니다.
Name: 1, dtype: object

In [17]:
user_data.iloc[56]

domitory                              1
age                                   2
student_id                            1
gender                                0
major                                 4
bedtime                               2
clean_duration                        0
smoke                                 0
alcohol                               0
mbti                               ISTP
one_sentence      대학교 가는 길마다 조깅을 좋아합니다.
Name: 57, dtype: object

In [18]:
user_data.iloc[9]

domitory                                          0
age                                               0
student_id                                        0
gender                                            1
major                                             3
bedtime                                           2
clean_duration                                    0
smoke                                             0
alcohol                                           0
mbti                                           ENFJ
one_sentence      헬스를 좋아하는데, 같이 운동할 수 있는 룸메이트 구합니다!
Name: 10, dtype: object